In [1]:
import torch

In [2]:

# Create a tensor of given shape
x = torch.randn(2, 3, 2)

# Create a tensor of given shape
y = torch.randn(2, 2, 4)

z = x @ y

print(z.shape)
print(z)

# Calculate the dot product of x and y
z = torch.bmm(x, y)
print(z.shape)

print(z)


torch.Size([2, 3, 4])
tensor([[[-0.7354, -0.2506,  1.0311,  0.8589],
         [ 0.3456, -0.7522, -1.0590, -0.3405],
         [-1.5162, -0.8362,  1.9151,  1.7940]],

        [[-1.4994, -1.6560, -3.3302, -2.9880],
         [ 0.5085,  0.1275, -0.0325,  0.0881],
         [ 0.3778,  1.2821,  3.1539,  2.5964]]])
torch.Size([2, 3, 4])
tensor([[[-0.7354, -0.2506,  1.0311,  0.8589],
         [ 0.3456, -0.7522, -1.0590, -0.3405],
         [-1.5162, -0.8362,  1.9151,  1.7940]],

        [[-1.4994, -1.6560, -3.3302, -2.9880],
         [ 0.5085,  0.1275, -0.0325,  0.0881],
         [ 0.3778,  1.2821,  3.1539,  2.5964]]])


In [3]:
# Create two tensors of the same shape (a, b, c)
a, b, c = 2, 3, 4  # Example dimensions
X = torch.randn(a, b, c)
Y = torch.randn(a, b, c)

# Compute Hadamard (element-wise) product
hadamard_product = X * Y

# Sum over all elements (generalized dot product)
GDT = hadamard_product.sum()

print("Generalized Dot Product (GDT):", GDT.item())  # Convert to scalar

GDT = torch.einsum('ijk,ijk->', X, Y)  # Summing over all elements
print("Generalized Dot Product (GDT):", GDT.item())

Generalized Dot Product (GDT): 10.08789348602295
Generalized Dot Product (GDT): 10.08789348602295


In [4]:
f = lambda x: x**2 - 1.5*x
df = lambda x: 2*x - 1.5

x = 1
def f_lin(h):
    # print(h, x)
    # print(f(x))
    # print(df(x))
    # print(h-x)
    return f(x) + df(x)*(h-x)

print(f(x + 0.001))
print(f_lin(x + 0.001))

-0.49949900000000014
-0.49950000000000006


In [5]:
x = 0.5
for i in range(10):
    h = 0.01    
    print(x)
    deriv = f_lin(x + h)
    print(deriv)
    x += deriv

0.5
-0.505
-0.0050000000000000044
-0.007574999999999994
-0.012575
0.003769130624999995
-0.008805869375000004
-0.0018897699895504324
-0.010695639364550437
0.0009439429609511673
-0.00975169640359927
-0.00047239373992511054
-0.01022409014352438
0.00023618543167899277
-0.009987904711845387
-0.00011814278593592561
-0.010106047497781313
5.908249274375402e-05
-0.010046965005037559
-2.9550286731964276e-05


In [9]:
from typing import Dict, Any, List
import numpy as np
import torch
import jax

from numpy.typing import NDArray

In [28]:
# Running a simple SDG in pytorch

def ls_sdg(
        X: NDArray,
        y: NDArray,
        parameters: Dict[str, Any]
) -> NDArray:
    
    epochs: int = parameters.get("epochs")
    learning_rate: float = parameters.get("lr")
    W: NDArray = parameters.get("parameters") 
    # For loop - can we do this in a Tensor
    for epoch in range(epochs):
        # Compute forward
        pred: NDArray = W @ X.T
        loss: float = ((y - pred)**2).mean()
        print(f"Epoch {epoch} - Loss: {loss}")
        # Compute backward
        grad: NDArray = -2 * (y - pred) @ X
        W -= learning_rate * grad
    return W


In [29]:
X: NDArray = np.random.randn(100, 2)
print(X.shape)

# We need to unpack the x1 and x2 into a single vector
x1: NDArray = X[:, 0]
x2: NDArray = X[:, 1]
y: NDArray = np.sin(x1) * np.cos(x2) + np.sin(0.5 * x1) * np.cos(0.5 * x2)
print(y.shape)

# Calculate the derivative of y
dy_dx1: NDArray = np.cos(x1) * np.cos(x2) - np.cos(0.5 * x1) * np.cos(0.5 * x2)
dy_dx2: NDArray = -np.sin(x1) * np.sin(x2) - 0.5 * np.sin(0.5 * x1) * np.sin(0.5 * x2)

parameters = {
    "epochs": 100,
    "lr": 0.01,
    "parameters": np.random.randn(10, 2)
}
ls_sdg(X, y, parameters)

(100, 2)
(100,)
Epoch 0 - Loss: 3.5335997774653216
Epoch 1 - Loss: 4.6450198025405545
Epoch 2 - Loss: 6.5141812208452246
Epoch 3 - Loss: 9.574774509849354
Epoch 4 - Loss: 14.517244618931322
Epoch 5 - Loss: 22.43974930029912
Epoch 6 - Loss: 35.087997090289804
Epoch 7 - Loss: 55.23627976834179
Epoch 8 - Loss: 87.2928085472242
Epoch 9 - Loss: 138.26138419313384
Epoch 10 - Loss: 219.26914857320605
Epoch 11 - Loss: 347.99362921131245
Epoch 12 - Loss: 552.5184272797243
Epoch 13 - Loss: 877.4584916862449
Epoch 14 - Loss: 1393.69092697865
Epoch 15 - Loss: 2213.8136631616408
Epoch 16 - Loss: 3516.70358375056
Epoch 17 - Loss: 5586.5303346164865
Epoch 18 - Loss: 8874.734874930784
Epoch 19 - Loss: 14098.490528907321
Epoch 20 - Loss: 22397.121269004834
Epoch 21 - Loss: 35580.59265361947
Epoch 22 - Loss: 56524.27223812105
Epoch 23 - Loss: 89796.05820612764
Epoch 24 - Loss: 142652.65535993912
Epoch 25 - Loss: 226622.29005120872
Epoch 26 - Loss: 360019.06019489886
Epoch 27 - Loss: 571937.3096624913
Ep

array([[ 3.17604541e+09, -1.26772695e+10],
       [-4.78320510e+09,  1.90922901e+10],
       [ 2.75112572e+09, -1.09811913e+10],
       [-3.51968871e+09,  1.40489309e+10],
       [-8.45596953e+08,  3.37522267e+09],
       [-5.47425472e+09,  2.18506330e+10],
       [ 2.84981651e+09, -1.13751182e+10],
       [-5.07532497e+09,  2.02582943e+10],
       [ 3.91897722e+09, -1.56427015e+10],
       [-2.90867977e+09,  1.16100725e+10]])